In [2]:
import tensorflow as tf

In [3]:

# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

Using TensorFlow backend.


In [ ]:
nltk.download('punkt')

In [4]:
# import our chat-bot intents file
import json
with open('intents edited.json') as json_data:
    intents = json.load(json_data)

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

74 documents
13 classes ['clients', 'company', 'contact', 'experience', 'goodbye', 'greeting', 'hours', 'location', 'name', 'projects', 'rates', 'services', 'thank you']
77 unique stemmed words ['a', 'about', 'address', 'am', 'answ', 'anyon', 'ar', 'brand', 'busy', 'bye', 'cal', 'can', 'cli', 'clientel', 'commun', 'company', 'contact', 'day', 'do', 'doe', 'experey', 'expery', 'expry', 'for', 'giv', 'good', 'goodby', 'guy', 'hav', 'height', 'hello', 'hi', 'hour', 'how', 'i', 'informt', 'is', 'lat', 'leav', 'loc', 'long', 'lot', 'me', 'nam', 'numb', 'of', 'off', 'op', 'pleas', 'pric', 'profil', 'project', 'quot', 'quota', 'rat', 'rec', 'recnt', 'see', 'serv', 'shar', 'should', 'tel', 'thank', 'the', 'ther', 'tim', 'to', 'typ', 'up', 'what', 'when', 'wher', 'which', 'who', 'work', 'yo', 'you']


In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [8]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer = sgd, metrics = ['accuracy'])

In [10]:
#fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size= 5, verbose=1)

Epoch 1/200
74/74 [==============================] - 0s 528us/step - loss: 0.0114 - accuracy: 1.0000
Epoch 2/200
74/74 [==============================] - 0s 487us/step - loss: 0.0299 - accuracy: 1.0000
Epoch 3/200
74/74 [==============================] - 0s 434us/step - loss: 0.0147 - accuracy: 1.0000
Epoch 4/200
74/74 [==============================] - 0s 433us/step - loss: 0.0713 - accuracy: 0.9730
Epoch 5/200
74/74 [==============================] - 0s 474us/step - loss: 0.0307 - accuracy: 0.9865
Epoch 6/200
74/74 [==============================] - 0s 432us/step - loss: 0.0301 - accuracy: 0.9865
Epoch 7/200
74/74 [==============================] - 0s 469us/step - loss: 0.0516 - accuracy: 0.9865
Epoch 8/200
74/74 [==============================] - 0s 427us/step - loss: 0.0364 - accuracy: 0.9865
Epoch 9/200
74/74 [==============================] - 0s 440us/step - loss: 0.0143 - accuracy: 1.0000
Epoch 10/200
74/74 [==============================] - 0s 454us/step - loss: 0.0898 - accura

74/74 [==============================] - 0s 469us/step - loss: 0.0210 - accuracy: 1.0000
Epoch 160/200
74/74 [==============================] - 0s 463us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 161/200
74/74 [==============================] - 0s 461us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 162/200
74/74 [==============================] - 0s 460us/step - loss: 0.0140 - accuracy: 1.0000
Epoch 163/200
74/74 [==============================] - 0s 450us/step - loss: 0.0027 - accuracy: 1.0000
Epoch 164/200
74/74 [==============================] - 0s 420us/step - loss: 0.0047 - accuracy: 1.0000
Epoch 165/200
74/74 [==============================] - 0s 448us/step - loss: 0.0096 - accuracy: 1.0000
Epoch 166/200
74/74 [==============================] - 0s 440us/step - loss: 0.0096 - accuracy: 1.0000
Epoch 167/200
74/74 [==============================] - 0s 467us/step - loss: 0.0105 - accuracy: 1.0000
Epoch 168/200
74/74 [==============================] - 0s 468us/step - loss: 0.1169 - a

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1


    return(np.array(bag))

In [12]:
p = bow("hi,how are you?", words)
print (p)
print (classes)

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1]
['clients', 'company', 'contact', 'experience', 'goodbye', 'greeting', 'hours', 'location', 'name', 'projects', 'rates', 'services', 'thank you']


In [13]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(model.predict(inputvar))

[[3.1490206e-09 6.0450872e-10 6.8379324e-10 6.3111219e-07 6.7849295e-09
  9.9999940e-01 1.0497726e-08 1.1271721e-08 5.6695506e-09 1.3760246e-10
  1.8300662e-08 8.3530388e-10 3.8495500e-09]]


In [49]:

# save model to file
pickle.dump(model, open("HC-assistant-model.pkl", "wb"))

In [50]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "HC-assistant-data.pkl", "wb" ) )

In [14]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.7
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [15]:
classify_local('What are your rates?')


[('rates', '0.9999987')]

In [16]:
def chat():
    print("Chat with HEIGHTS COMMUNICATIONS bot assistant! (type quit to exit)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
                
        input_data = pd.DataFrame([bow(inp, words)], dtype=float, index=['input'])
        results = model.predict([input_data])[0]
        results_index = np.argmax(results)
        tag = classes[results_index]
        
        if results[results_index] > 0.7:
            for tg in intents["intents"]:
                if tg['tag'] == tag:
                    responses = tg["responses"]
            print("Assistant:", random.choice(responses))
        else:
            print("Assistant:","I am sorry. I am not aware of what you are asking. Please call us at +923232779999")
chat()

Chat with HEIGHTS COMMUNICATIONS bot assistant! (type quit to exit)
You: quit


In [17]:
import time
saved_model_path = "./hc_model.h5".format(int(time.time()))

model.save(saved_model_path)

In [18]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./